In [3]:
from PIL import Image

# Load TIFF image and convert to grayscale
tiff_path = 'data/images/0.tiff'
png_path = 'output_image.png'

with Image.open(tiff_path) as img:
    gray_img = img.convert('L')  # 'L' mode is grayscale
    gray_img.save(png_path)

print(f"Converted {tiff_path} to grayscale PNG saved as {png_path}")


DecompressionBombError: Image size (813957120 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.